# 제2 고지 : 자연스러운  코드로 
## STEP 18 : 메모리 절약 모드

이전 단계에서는 파이썬의 메모리 관리 방식에 대해 알아봤다. 이번 단계에서는 Dezero의 메모리 사용을 개선할 수 있는 구조 두가지를 도입한다.

1. 역전파시 불필요한 결과를 보관하지 않고 즉시 삭제

2. `역전파가 필요 없는 경우용 모드` 제공



### 18.1 필요 없는 미분값 삭제 
현재 DeZero에서는 **모든 변수가 미분값을 저장**하고 있다.  
그러나 많은 경우, 특히 머신러닝에서 구하고 싶은 미분값은 사용자가 제공한 변수 `x0`,`x1` 의 미분값뿐일 때가 대부분이고, `y`,`t`와 같은 **중간 변수의 미분값은 필요로 하지 않는다.**     
또한, `학습(training)`에는 **미분값이 필요**하지만, `추론(inference)` 시에는 **단순히 순전파만 진행하여 미분값이 필요없는 경우**도 있다. 
$$
x= x_0+x_1 \\
t = x_0+x_1 \\
y = x_0+t \\
\nabla_xf = \begin{bmatrix}\frac{\partial y}{\partial x_0} & \frac{\partial y}{\partial x_1}\end{bmatrix} \\
\begin{aligned}
&\Rightarrow \frac{\partial y}{\partial x_0} =\frac{\partial }{\partial x_0}(x_0+t)=1 + \frac{\partial t}{\partial x_0} = 1 + \frac{\partial}{\partial x_0}(x_0+x_1)  = 2 \\
&\Rightarrow \frac{\partial y}{\partial x_1} =\frac{\partial }{\partial x_1}(x_0+t)=\frac{\partial t}{\partial x_1} = \frac{\partial}{\partial x_1}(x_0+x_1)  = 1
\end{aligned}
$$

이해를 돕기 위해 현재의 DeZero를 예시를 통해 살펴보면, 사용자가 제공한 변수는 `x0`, `x1` 이고 `y`,`t` 는 계산 결과로 만들어진다. 그리고, `y.backward()`를 통해 역전파를 실행하면 모든 변수가 미분 결과를 메모리에 유지한다. 

```python
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0,x1)
y = add(x0,t)

y.backward()
print("현재 모든 변수가 미분값 저장:")
print(y.grad,t.grad)
print(x0.grad,x1.grad)
'''
현재 모든 변수가 미분값 저장:
1.0 1.0
2.0 1.0
'''
``` 

그래서 메모리 개선을 위해 `중간 변수에 대해서는 미분값을 제거하는 모드`를 선택할 수 있는 `retain_grad` 를 추가한다.  
만약 `retain_grad` 가 `True` 라면 **중간 변수 미분값을 유지**, `False` 라면 **중간 변수의 모든 미분값을** `None`으로 재설정한다.


In [4]:
import weakref
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None
    ###################################
    def backward(self,retain_grad=False):  # `retain_grad` 추가 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)
            ##################################
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref이기 때문에 y()로 호출
            ##################################
    #################################################

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0,x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        # 역전파시 , 입력이 1개 , 출력이 2개 
        return gy,gy 
    
def add(x0,x1):
    return Add()(x0,x1)

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)

In [22]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0,x1)
y = add(x0,t)

y.backward()
print("중간 변수 미분값은 제거:")
print(y.grad,t.grad)
print(x0.grad,x1.grad)


DeZero : 중간 변수 미분값은 제거:
None None
2.0 1.0


### 18.2 Function 클래스 복습 

**역전파를 하기 위해서는 순전파의 계산 결과값이 필요하기 때문에 순전파 때 결과값을 기억**해둬야 한다.  
이를 구현하기 위해 현재의 DeZero는 인스턴수 변수 `self.inputs` 를 참조하여 값을 저장해두고 있는데, 이는 `__call__()`를 **빠져나간 뒤에도 메모리에 남아 있다.**

```python
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        ########################
        # 현재는 `인스턴수 변수`로 참조하여, `__call__()` 를 빠져나간뒤에도 메모리에 남아 있다. 
        self.inputs = inputs
        ########################
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()
```

하지만, 때로는 **미분값이 필요없는 경우라면(=순전파만 진행하는 경우라면)**, 중간 계산 결과값을 굳이 저장할 필요가 없으며, 계산의 연결 또한 만들 필요가 없다.

### 18.3 Config 클래스를 활용한 모드 전환 

앞서 언급한 **중간 계산 결과값을 저장할 필요가 없는 경우** 를 반영하기 위해, `Config` 클래스를 이용해 모드 전환을 쉽게 할 수 있도록 한다. 

```python
class Config :
    enable_backprop=True # `역전파가 가능한지의 여부`로 True 라면 역전파 활성모드
```

위의 코드에서 주목할 것은 두가지이다.
- `enable_backprop` 이 `True` 이면, **역전파 활성모드**
- 설정 데이터인 `enable_backprop`을 **인스턴스화하지 않고(=인스턴스 변수가 아닌) 클래스 상태(=클래스 변수)로 유지**



In [6]:
class Config :
    enable_backprop=True # `역전파가 가능한지의 여부`로 True 라면 역전파 활성모드
    
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        ########################
        if Config.enable_backprop:   #역전파 활성모드 일 경우에만 
            self.generation = max([x.generation for x in inputs]) # 1. 세대 설정
            for output in outputs:
                output.set_creator(self) # 2. 연결 설정 
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        ########################
        
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 18.4 모드 전환 
이제는 역전파 활성/비활성을 구분 지을 수 있으므로 이를 활용하면 다음과 같이 모드를 전환할 수 있다.

In [29]:
# 역전파 활성화
Config.enable_backprop=True
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
y.backward()


# 역전파 비활성화
Config.enable_backprop=False
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))


### 18.5 with 문을 활용한 모드 전환

파이썬에서 가장 많이 사용되는 `컨텍스트 매니저(Context Manager)` 인 `with` 를 이용하여 **모드 전환이 유연하게 가능하도록 코드를 구현**한다.  
주로 **리소스 관리와 관련하여 많이 사용**되는데, 대표적으로 `파일 열기/닫기`, `DB 연결/해제` 등이 있다. 
아래의 예시에서, `with`문을 활용하면 `f.close()`를 신경쓰지 않고, **예외 또는 오류가 발생**하더라도 항상  **후처리를 자동으로 실행**하는 것을 확인할 수 있다.

```python
f = open("sample.txt","w")
try : 
    f.write("hello world!")
finally :
    f.close()

## with 문을 활용
with open("sample.txt","w") as f :
    f.write("hello world!")
```

이를 구현하기 위해서는 `contextlib` 모듈을 활용하면 가장 쉽게 구현할 수 있다. (물론 `class`를 활용하여 직접 매직 메서드 `__enter__()`, `__exit__()` 를 구현하여도 상관없다. )

```python
import contextlib

@contextlib.contextmanager #_GeneratorContextManager 
def config_test():
    print("start")  # 전처리
    try:
        yield 
    finally:
        print("done")  # 후처리

with config_test() as f :
    print("process..")
"""
start
process..
done
"""
```

In [8]:
import contextlib

class Config :
    enable_backprop=True # `역전파가 가능한지의 여부`로 True 라면 역전파 활성모드
    
    
@contextlib.contextmanager # _GeneratorContextManager 
def using_config(name,value):
    old_value = getattr(Config,name)
    setattr(Config,name,value)
    try : 
        yield
    finally:
        setattr(Config,name,old_value) #  기존 설정값으로 원복 

def no_grad():
    ## 매번 작성하기 귀찮으므로 helper 함수 작성 
    return using_config("enable_backprop",False)


In [18]:
# Dezero ~ PyTorch 

## DeZero
with using_config("enable_backprop",False):
    x = Variable(np.array(2.0))
    y= square(x)

with no_grad():
    x = Variable(np.array(2.0))
    y= square(x)
    
## PyTorch
import torch 
with torch.no_grad():
    x = torch.tensor([2.0])
    y = x**2